In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict

import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interactive, FloatSlider

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, confusion_matrix, precision_score, recall_score, plot_roc_curve
from sklearn.model_selection import train_test_split, GridSearchCV, KFold

from joblib import dump, load
import os

import eda

import psycopg2 as pg
import sql_fish

RANDOM = 42
connection = sql_fish.fishing_database()

In [3]:
query = "SELECT is_fishing, COUNT(*) as record_count FROM fishingfeatures GROUP BY is_fishing ORDER BY is_fishing;"

pd.io.sql.read_sql(query, connection)

,is_fishing,record_count
0,-1.000000,28027543
1,0.000000,295979
2,0.166667,12
3,0.250000,670
4,0.333333,4096
5,0.400000,9
6,0.666667,4806
7,0.750000,752
8,0.800000,33
9,1.000000,247498


In [4]:
query = "SELECT gear_type, COUNT(*) AS record_count FROM fishingfeatures GROUP BY gear_type ORDER BY record_count DESC;"

gear_record_summary_df = pd.io.sql.read_sql(query, connection)

gear_record_summary_df

,gear_type,record_count
0,drifting_longlines,13968727
1,unknown,6811552
2,trawlers,4369101
3,fixed_gear,1559137
4,purse_seines,1545323
5,trollers,166243
6,pole_and_line,161315


In [5]:
query = "SELECT gear_type, COUNT(DISTINCT mmsi) AS ship_count FROM fishingfeatures GROUP BY gear_type ORDER BY ship_count DESC;"

gear_ship_summary_df = pd.io.sql.read_sql(query, connection)

gear_ship_summary_df

,gear_type,ship_count
0,unknown,120
1,drifting_longlines,110
2,trawlers,49
3,fixed_gear,36
4,purse_seines,28
5,pole_and_line,6
6,trollers,5


In [ ]:
query = "SELECT * FROM fishingfeatures WHERE is_fishing != -1;"

fishing_raw_df = pd.io.sql.read_sql(query, connection)

fishing_raw_df